# code to import predictions for Types having 7 days predictions:



1 Wind
2 Solar
17 WindOffshore
20 Nuclear
21 WastePower
26 OtherPower



Import required dependencies

In [6]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data
import time


# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

load config files

In [7]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

Config parameters for API calls

In [8]:
# Load config values
config = load_config(CONFIG_FILE)
# print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}

Make API calls in loop

In [10]:
def fetch_and_save_data():
    current_datetime = datetime.utcnow()
    future_datetime = current_datetime + timedelta(days=8)

    current_date_str = current_datetime.strftime("%Y-%m-%d")
    future_date_str = future_datetime.strftime("%Y-%m-%d")

    for gen_type in n_values:
        api_params = {
            "point": "0",
            "type": gen_type,
            "granularity": "5",  # hourly
            "granularitytimezone": "1",
            "classification": "1",  # Forecast
            "activity": "1",
            "validfrom[before]": future_date_str,
            "validfrom[after]": current_date_str,
            "page": 1
        }

        full_data = []

        response = requests.get(API_URL, params=api_params, headers=api_headers)
        if response.status_code == 200:
            data = response.json()
            last_page_url = data.get("hydra:view", {}).get("hydra:last", None)
            last_page_num = int(last_page_url.split("page=")[-1]) if last_page_url else 1
        else:
            print(f"Error {response.status_code}: {response.text}")
            return

        for page in range(1, last_page_num + 1):
            api_params["page"] = page
            response = requests.get(API_URL, params=api_params, headers=api_headers)
            if response.status_code == 200:
                json_dict = response.json()
                if "hydra:member" in json_dict:
                    full_data.extend(json_dict["hydra:member"])
            else:
                print(f"Error on page {page}: {response.status_code}")
                break

        df = pd.DataFrame(full_data)
        df['current_datetime'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        output_file = "../Data/appended_NED_preds.csv"
        output_dir = os.path.dirname(output_file)
        os.makedirs(output_dir, exist_ok=True)

        # Append to the CSV file
        if not os.path.exists(output_file):
            df.to_csv(output_file, index=False)  # Write header if file doesn't exist
        else:
            df.to_csv(output_file, mode='a', header=False, index=False)  # Append without header

        print(f"Data collection complete! Appended {len(df)} records of Type-{gen_type} to {output_file}")


In [5]:

from datetime import datetime, timedelta
import time

n_values = [1,2,17,20,21,26]  # Selected Types of Electricity Sources (NED.nl)

while True:
    # Update current time and week ahead time using UTC
    current_datetime = datetime.utcnow()
    future_datetime = current_datetime + timedelta(days=8)

    # Replace the datetime values with formatted strings only showing YYYY-MM-DD
    current_date_str = current_datetime.strftime("%Y-%m-%d")
    future_date_str = future_datetime.strftime("%Y-%m-%d")

    # Loop through the n values
    for gen_type in n_values:
        api_params = {
            "point": "0",
            "type": gen_type,
            "granularity": "5",  # hourly
            "granularitytimezone": "1",
            "classification": "1",  # Forecast
            "activity": "1",
            "validfrom[before]": future_date_str,
            "validfrom[after]": current_date_str,
            "page": 1
        }

        # Initialize storage for full dataset
        full_data = []

        # First API request to determine the last page
        response = requests.get(API_URL, params=api_params, headers=api_headers)
        if response.status_code == 200:
            data = response.json()
            last_page_url = data.get("hydra:view", {}).get("hydra:last", None)

            if last_page_url:
                last_page_num = int(last_page_url.split("page=")[-1])
            else:
                print("Warning: Could not determine last page. Defaulting to 1.")
                last_page_num = 1
        else:
            print(f"Error {response.status_code}: {response.text}")
            continue

        # Iterate through pages
        for page in range(1, last_page_num + 1):
            api_params["page"] = page
            response = requests.get(API_URL, params=api_params, headers=api_headers)

            if response.status_code == 200:
                json_dict = response.json()
                if "hydra:member" in json_dict:
                    full_data.extend(json_dict["hydra:member"])
            else:
                print(f"Error on page {page}: {response.status_code}")
                break

        # After collecting all pages, convert the data to a DataFrame and add current_datetime column
        df = pd.DataFrame(full_data)
        df['current_datetime'] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
        file_name = f"power-gen-pred-{gen_type}-{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"
        # Ensure the directory exists
        output_dir = "../Data/powergen_preds"
        os.makedirs(output_dir, exist_ok=True)
        file_path = os.path.join(output_dir, file_name)
        df.to_csv(file_path, index=False)

        print(f"Data collection complete! Saved {len(df)} records of Type-{gen_type} to csv")
        print(f"Saved file: {file_name}")

    # Wait exactly one hour before fetching data again
    print("Waiting for the next fetch in 4 hours...")
    time.sleep(4*3600)


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_datetime = datetime.utcnow()
/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df['current_datetime'] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  file_name = f"power-gen-pre

Data collection complete! Saved 186 records of Type-1 to csv
Saved file: power-gen-pred-1-20250407_1840.csv


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df['current_datetime'] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  file_name = f"power-gen-pred-{gen_type}-{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"


Data collection complete! Saved 186 records of Type-2 to csv
Saved file: power-gen-pred-2-20250407_1840.csv


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df['current_datetime'] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_33949/1366933331.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  file_name = f"power-gen-pred-{gen_type}-{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"


...

Data collection complete! Saved 185 records of Type-20 to csv
Saved file: power-gen-pred-20-20250407_1713.csv


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_18431/1366933331.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df['current_datetime'] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_18431/1366933331.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  file_name = f"power-gen-pred-{gen_type}-{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"


Data collection complete! Saved 185 records of Type-21 to csv
Saved file: power-gen-pred-21-20250407_1713.csv


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_18431/1366933331.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df['current_datetime'] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_18431/1366933331.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  file_name = f"power-gen-pred-{gen_type}-{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"


Data collection complete! Saved 185 records of Type-26 to csv
Saved file: power-gen-pred-26-20250407_1713.csv
Waiting for the next fetch in 4 hours...


# to add function to regularly make API calls, also need to add a timestamp

In [3]:
# Iterate through all CSV files in the directory
for file_name in os.listdir("../Data"):
    if file_name.endswith(".csv"):
        file_path = os.path.join(data_dir, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Concatenate all dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined dataframe to a single CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined CSV file saved to: {output_file}")

NameError: name 'os' is not defined